In [15]:
### import all the dependencies 
import requests
import csv
import json
import pandas as pd
# import requests, json
import re

In [16]:
### Main Program starts here  - Now we will read data form CSV

external_customers_df = pd.read_csv("../data/external_customers.csv", low_memory=False)
print('Shape of exports read from CSV : {}'.format(external_customers_df.shape))

Shape of exports read from CSV : (53906, 16)


In [17]:
##List the names of columns 
external_customers_df.columns


Index(['Unnamed: 0', 'Domain', 'UserEmail', 'serviceName', 'UserId', 'IndvId',
       'SessionId', 'PageUrl', 'PageRefererUrl', 'PageBrowser', 'PageDevice',
       'PageDuration', 'PageActiveDuration', 'PageLatLong', 'EventStart',
       'EventType'],
      dtype='object')

In [18]:
len(external_customers_df.columns)

16

In [19]:
## Data Cleaning and filtering 
#dropping an Column not needed 
print('Number of columns before :{}'.format(len(external_customers_df.columns)))
external_customers_df.drop('Unnamed: 0', axis=1, inplace=True)
print('Number of columns after :{}'.format(len(external_customers_df.columns)))

Number of columns before :16
Number of columns after :15


In [20]:
external_customers_df.head()   


,Domain,UserEmail,serviceName,UserId,IndvId,SessionId,PageUrl,PageRefererUrl,PageBrowser,PageDevice,PageDuration,PageActiveDuration,PageLatLong,EventStart,EventType
0,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.cloud.dataguru.com/li/,https://console.cloud.dataguru.com/bin/gateway...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:10:55.257Z,navigate
1,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.cloud.dataguru.com/li/,https://console.cloud.dataguru.com/bin/gateway...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:10:56.231Z,load
2,Company1.com,chad.tew@Company1.com,Data Logging,5650932471693312,5650932471693312,5629499534213120,https://www.cloud.dataguru.com/li/,https://console.cloud.dataguru.com/bin/gateway...,Chrome,Desktop,20030,3459,"33.3335,-86.9591",2018-09-08T23:11:15.045Z,click
3,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.cloud.dataguru.com/catalog/#/library,https://console.cloud.dataguru.com/bin/gateway...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:03.950Z,navigate
4,Company2.com,mike.hutt@Company2.com,SDK,5746562451898368,5767773206609920,5629499534213120,https://www.cloud.dataguru.com/catalog/#/library,https://console.cloud.dataguru.com/bin/gateway...,Chrome,Desktop,2038981,249088,"48.8628,2.3292",2018-09-10T09:33:04.026Z,load
